In [58]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers.pipelines.pt_utils import KeyDataset
#from datasets import load_dataset
import matplotlib.pyplot as plt
import torch 
import torch.nn.functional as F
import io
import seaborn as sns
import pandas as pd
import numpy as np
import glob
import os
import re
import datetime

In [59]:
# setting the path for joining multiple files
files = os.path.join("../Raw(uncleaned) csv files", "Amazon.co.uk_ Auto_all***.xlsx")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df = pd.concat(map(pd.read_excel, files), ignore_index=True)
print(df)

Resultant CSV after joining all CSV files at a particular location...
                                                    Text  \
0                                   Bosch Headlight Bulb   
1                                   Bosch Headlight Bulb   
2      Dr. Beckmann Carpet Stain Remover | Removes ne...   
3                                   Bosch Headlight Bulb   
4      Dr. Beckmann Carpet Stain Remover | Removes ne...   
...                                                  ...   
75010                               RainX Rain Repellant   
75011                               RainX Rain Repellant   
75012                               RainX Rain Repellant   
75013                               RainX Rain Repellant   
75014                               RainX Rain Repellant   

                                                   Title             Name  \
0                                      Lasted 10 minutes              Baz   
1                                      Nice bright ligh

In [60]:
df.head()

,Text,Title,Name,aiconalt,View,State,View1,asizebase
0,Bosch Headlight Bulb,Lasted 10 minutes,Baz,1.0 out of 5 stars,Reviewed in the United Kingdom on 13 November ...,Verified Purchase,Product arrived in hassle free packaging. One...,14 people found this helpful
1,Bosch Headlight Bulb,Nice bright light,Maru Marek,5.0 out of 5 stars,Reviewed in the United Kingdom on 21 November ...,Verified Purchase,NaN,6 people found this helpful
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,Brilliant product,Kindle Customer,5.0 out of 5 stars,Reviewed in the United Kingdom on 13 January 2020,Verified Purchase,I have used this product for a year or two and...,10 people found this helpful
3,Bosch Headlight Bulb,Good all round improvement to light throw and ...,LondonRascal,4.0 out of 5 stars,Reviewed in the United Kingdom on 24 February ...,Verified Purchase,These are based on Halogen technology so anyon...,10 people found this helpful
4,Dr. Beckmann Carpet Stain Remover | Removes ne...,"Amazing product, removed horrible coffee stain!",Ali,4.0 out of 5 stars,Reviewed in the United Kingdom on 3 September ...,Verified Purchase,Great product. Having tried many other stain r...,42 people found this helpful


In [61]:
df.rename(columns = {"Text":"Product", "Title":"Title of Review", "View":"Location and Date", "aiconalt":"Individual Review", "View1": "Review", "asizebase": "Helpful"}, inplace= True)

In [62]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful
0,Bosch Headlight Bulb,Lasted 10 minutes,Baz,1.0 out of 5 stars,Reviewed in the United Kingdom on 13 November ...,Verified Purchase,Product arrived in hassle free packaging. One...,14 people found this helpful
1,Bosch Headlight Bulb,Nice bright light,Maru Marek,5.0 out of 5 stars,Reviewed in the United Kingdom on 21 November ...,Verified Purchase,NaN,6 people found this helpful
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,Brilliant product,Kindle Customer,5.0 out of 5 stars,Reviewed in the United Kingdom on 13 January 2020,Verified Purchase,I have used this product for a year or two and...,10 people found this helpful
3,Bosch Headlight Bulb,Good all round improvement to light throw and ...,LondonRascal,4.0 out of 5 stars,Reviewed in the United Kingdom on 24 February ...,Verified Purchase,These are based on Halogen technology so anyon...,10 people found this helpful
4,Dr. Beckmann Carpet Stain Remover | Removes ne...,"Amazing product, removed horrible coffee stain!",Ali,4.0 out of 5 stars,Reviewed in the United Kingdom on 3 September ...,Verified Purchase,Great product. Having tried many other stain r...,42 people found this helpful


In [63]:
df.isna().sum()

Product                  0
Title of Review      23671
Name                    60
Individual Review       57
Location and Date       57
State                 1841
Review                 117
Helpful              64155
dtype: int64

In [64]:
df.drop(columns=["State", "Title of Review", "State", "Name"], inplace=True)

In [65]:
df.dropna(subset=["Review"], inplace=True)

In [66]:
df.isna().sum()

Product                  0
Individual Review        0
Location and Date        0
Review                   0
Helpful              64051
dtype: int64

In [67]:
df.describe()

,Product,Individual Review,Location and Date,Review,Helpful
count,74898,74898,74898,74898,10847
unique,56,5,13488,67929,185
top,Ring RAC610 In-Car Air Compressor for Cars and...,5.0 out of 5 stars,Reviewed in the United Kingdom on 13 December ...,Good,One person found this helpful
freq,5003,53003,103,430,5648


In [68]:
df[["Individual Review"]] = df[["Individual Review"]].apply(lambda x: x.str.rstrip(' out of 5 stars'))

In [69]:
df =  df.astype(str)


In [70]:
df.head()

,Product,Individual Review,Location and Date,Review,Helpful
0,Bosch Headlight Bulb,1.0,Reviewed in the United Kingdom on 13 November ...,Product arrived in hassle free packaging. One...,14 people found this helpful
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,Reviewed in the United Kingdom on 13 January 2020,I have used this product for a year or two and...,10 people found this helpful
3,Bosch Headlight Bulb,4.0,Reviewed in the United Kingdom on 24 February ...,These are based on Halogen technology so anyon...,10 people found this helpful
4,Dr. Beckmann Carpet Stain Remover | Removes ne...,4.0,Reviewed in the United Kingdom on 3 September ...,Great product. Having tried many other stain r...,42 people found this helpful
5,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,Reviewed in the United Kingdom on 17 June 2018,Worked well. It won’t let me put up a before p...,49 people found this helpful


In [71]:
df['Location and Date'] = df['Location and Date'].apply(lambda x: x.lstrip('Reviewed in the '))

In [72]:
df.head()

,Product,Individual Review,Location and Date,Review,Helpful
0,Bosch Headlight Bulb,1.0,United Kingdom on 13 November 2019,Product arrived in hassle free packaging. One...,14 people found this helpful
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom on 13 January 2020,I have used this product for a year or two and...,10 people found this helpful
3,Bosch Headlight Bulb,4.0,United Kingdom on 24 February 2021,These are based on Halogen technology so anyon...,10 people found this helpful
4,Dr. Beckmann Carpet Stain Remover | Removes ne...,4.0,United Kingdom on 3 September 2018,Great product. Having tried many other stain r...,42 people found this helpful
5,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom on 17 June 2018,Worked well. It won’t let me put up a before p...,49 people found this helpful


In [73]:
df["Helpful"] = df['Helpful'].apply(lambda x: x.replace(',',''))

In [74]:
df.head()

,Product,Individual Review,Location and Date,Review,Helpful
0,Bosch Headlight Bulb,1.0,United Kingdom on 13 November 2019,Product arrived in hassle free packaging. One...,14 people found this helpful
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom on 13 January 2020,I have used this product for a year or two and...,10 people found this helpful
3,Bosch Headlight Bulb,4.0,United Kingdom on 24 February 2021,These are based on Halogen technology so anyon...,10 people found this helpful
4,Dr. Beckmann Carpet Stain Remover | Removes ne...,4.0,United Kingdom on 3 September 2018,Great product. Having tried many other stain r...,42 people found this helpful
5,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom on 17 June 2018,Worked well. It won’t let me put up a before p...,49 people found this helpful


In [75]:
df["Date"] = df["Location and Date"]

In [76]:
df["Date"] = df["Date"].apply(lambda x: x.split(' on ')[-1].strip())

In [77]:
df.head()

,Product,Individual Review,Location and Date,Review,Helpful,Date
0,Bosch Headlight Bulb,1.0,United Kingdom on 13 November 2019,Product arrived in hassle free packaging. One...,14 people found this helpful,13 November 2019
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom on 13 January 2020,I have used this product for a year or two and...,10 people found this helpful,13 January 2020
3,Bosch Headlight Bulb,4.0,United Kingdom on 24 February 2021,These are based on Halogen technology so anyon...,10 people found this helpful,24 February 2021
4,Dr. Beckmann Carpet Stain Remover | Removes ne...,4.0,United Kingdom on 3 September 2018,Great product. Having tried many other stain r...,42 people found this helpful,3 September 2018
5,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom on 17 June 2018,Worked well. It won’t let me put up a before p...,49 people found this helpful,17 June 2018


In [78]:
df["Location and Date"] = df["Location and Date"].apply(lambda x: x.split(' on ')[0].strip())

In [79]:
df.head()

,Product,Individual Review,Location and Date,Review,Helpful,Date
0,Bosch Headlight Bulb,1.0,United Kingdom,Product arrived in hassle free packaging. One...,14 people found this helpful,13 November 2019
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,I have used this product for a year or two and...,10 people found this helpful,13 January 2020
3,Bosch Headlight Bulb,4.0,United Kingdom,These are based on Halogen technology so anyon...,10 people found this helpful,24 February 2021
4,Dr. Beckmann Carpet Stain Remover | Removes ne...,4.0,United Kingdom,Great product. Having tried many other stain r...,42 people found this helpful,3 September 2018
5,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,Worked well. It won’t let me put up a before p...,49 people found this helpful,17 June 2018


In [80]:
df.rename(columns={"Location and Date": "Location"}, inplace = True)

In [81]:
df.head()

,Product,Individual Review,Location,Review,Helpful,Date
0,Bosch Headlight Bulb,1.0,United Kingdom,Product arrived in hassle free packaging. One...,14 people found this helpful,13 November 2019
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,I have used this product for a year or two and...,10 people found this helpful,13 January 2020
3,Bosch Headlight Bulb,4.0,United Kingdom,These are based on Halogen technology so anyon...,10 people found this helpful,24 February 2021
4,Dr. Beckmann Carpet Stain Remover | Removes ne...,4.0,United Kingdom,Great product. Having tried many other stain r...,42 people found this helpful,3 September 2018
5,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,Worked well. It won’t let me put up a before p...,49 people found this helpful,17 June 2018


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74898 entries, 0 to 75014
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Product            74898 non-null  object
 1   Individual Review  74898 non-null  object
 2   Location           74898 non-null  object
 3   Review             74898 non-null  object
 4   Helpful            74898 non-null  object
 5   Date               74898 non-null  object
dtypes: object(6)
memory usage: 4.0+ MB


In [83]:
df[["day","month", "year"]] = df["Date"].apply(lambda x: pd.Series(x.split(' ')))

In [84]:
df["Date"] = pd.to_datetime(df['year'].astype(str)  + df['month'] + df["day"], format='%Y%B%d')

In [85]:
df.head()

,Product,Individual Review,Location,Review,Helpful,Date,day,month,year
0,Bosch Headlight Bulb,1.0,United Kingdom,Product arrived in hassle free packaging. One...,14 people found this helpful,2019-11-13,13,November,2019
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,I have used this product for a year or two and...,10 people found this helpful,2020-01-13,13,January,2020
3,Bosch Headlight Bulb,4.0,United Kingdom,These are based on Halogen technology so anyon...,10 people found this helpful,2021-02-24,24,February,2021
4,Dr. Beckmann Carpet Stain Remover | Removes ne...,4.0,United Kingdom,Great product. Having tried many other stain r...,42 people found this helpful,2018-09-03,3,September,2018
5,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,Worked well. It won’t let me put up a before p...,49 people found this helpful,2018-06-17,17,June,2018


In [86]:
df.drop(columns = ["day", "month", "year"], inplace= True)

In [87]:
df["Helpful"].value_counts()

nan                              64051
One person found this helpful     5648
2 people found this helpful       1630
3 people found this helpful        831
4 people found this helpful        496
                                 ...  
215 people found this helpful        1
152 people found this helpful        1
429 people found this helpful        1
147 people found this helpful        1
301 people found this helpful        1
Name: Helpful, Length: 186, dtype: int64

In [88]:
df["Helpful"] = df["Helpful"].apply(lambda x: x.rstrip('people found this helpful'))

In [89]:
df["Helpful"].value_counts()

na         64051
One per     5648
2           1630
3            831
4            496
           ...  
471            1
189            1
98             1
135            1
259            1
Name: Helpful, Length: 186, dtype: int64

In [90]:
df["Helpful"].replace(['One per', "na"], [1, 0], inplace = True)

In [91]:
df.head()

,Product,Individual Review,Location,Review,Helpful,Date
0,Bosch Headlight Bulb,1.0,United Kingdom,Product arrived in hassle free packaging. One...,14,2019-11-13
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,I have used this product for a year or two and...,10,2020-01-13
3,Bosch Headlight Bulb,4.0,United Kingdom,These are based on Halogen technology so anyon...,10,2021-02-24
4,Dr. Beckmann Carpet Stain Remover | Removes ne...,4.0,United Kingdom,Great product. Having tried many other stain r...,42,2018-09-03
5,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,Worked well. It won’t let me put up a before p...,49,2018-06-17


In [92]:
df[["Product", "Location", "Review"]].astype(str).dtypes
df[["Individual Review"]].astype(float)
df[["Helpful"]].astype(int)

,Helpful
0,14
2,10
3,10
4,42
5,49
...,...
75010,0
75011,0
75012,2
75013,1


In [93]:
df["Location"].value_counts()

United Kingdom    51325
Germany            7422
Italy              3694
France             2858
United States      2667
Spain              2344
Canada             2224
India              1684
Mexico              386
Japan               108
Australia           103
Brazil               48
Netherlands          21
Sweden                9
Singapore             4
Poland                1
Name: Location, dtype: int64

In [94]:
(df == "NaN").all(axis=0)

Product              False
Individual Review    False
Location             False
Review               False
Helpful              False
Date                 False
dtype: bool

In [95]:
#greater than the start date and smaller than the end date
df_covid_dates = df[(df["Date"] > "2019-12-01") & (df['Date'] <= "2020-08-01")]

In [96]:
(df_covid_dates["Review"] == "NaN").value_counts()

False    9770
Name: Review, dtype: int64

In [97]:
df_covid_dates["Review"].value_counts()

Good                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               62
Great product                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [98]:
df_covid_dates["Location"].value_counts()

United Kingdom    6799
Germany           1012
Italy              468
France             375
Canada             353
Spain              258
United States      222
India              199
Mexico              60
Japan               16
Australia            7
Netherlands          1
Name: Location, dtype: int64

### Drop non English speaking countries

In [99]:
df_covid_dates = df_covid_dates.loc[df_covid_dates["Location"].isin(["United Kingdom", "United States", "Canada", "India", "Australia"])]

In [100]:
df_covid_dates["Location"].value_counts()

United Kingdom    6799
Canada             353
United States      222
India              199
Australia            7
Name: Location, dtype: int64

In [101]:
df_covid_dates.dtypes


Product                      object
Individual Review            object
Location                     object
Review                       object
Helpful                      object
Date                 datetime64[ns]
dtype: object

In [102]:
df_covid_dates

,Product,Individual Review,Location,Review,Helpful,Date
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,I have used this product for a year or two and...,10,2020-01-13
15,Bosch Headlight Bulb,4.0,United Kingdom,Bought these after buying so called brighter l...,6,2020-04-17
18,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,Was so terrified cos I spilled an unbelievably...,86,2020-05-30
31,Amazon Basics Microfibre Cleaning Cloths Pack ...,5.0,United Kingdom,I often pick up Amazon branded products for ho...,5,2020-06-14
35,Amazon Basics Microfibre Cleaning Cloths Pack ...,4.0,United Kingdom,"Summary: Excellent Quality, a bit pricey.\n\nP...",0,2020-05-30
...,...,...,...,...,...,...
74935,RainX Rain Repellant,5.0,United Kingdom,Works perfect for my windscreen.,0,2019-12-22
74941,RainX Rain Repellant,5.0,United Kingdom,V Good,0,2019-12-08
74942,RainX Rain Repellant,5.0,United Kingdom,Good,0,2019-12-06
74945,RainX Rain Repellant,5.0,United Kingdom,"Have used this product before, it is excellent",0,2020-04-22


In [103]:
df_covid_dates.reset_index(drop=True, inplace=True)
df_covid_dates

,Product,Individual Review,Location,Review,Helpful,Date
0,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,I have used this product for a year or two and...,10,2020-01-13
1,Bosch Headlight Bulb,4.0,United Kingdom,Bought these after buying so called brighter l...,6,2020-04-17
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,5.0,United Kingdom,Was so terrified cos I spilled an unbelievably...,86,2020-05-30
3,Amazon Basics Microfibre Cleaning Cloths Pack ...,5.0,United Kingdom,I often pick up Amazon branded products for ho...,5,2020-06-14
4,Amazon Basics Microfibre Cleaning Cloths Pack ...,4.0,United Kingdom,"Summary: Excellent Quality, a bit pricey.\n\nP...",0,2020-05-30
...,...,...,...,...,...,...
7575,RainX Rain Repellant,5.0,United Kingdom,Works perfect for my windscreen.,0,2019-12-22
7576,RainX Rain Repellant,5.0,United Kingdom,V Good,0,2019-12-08
7577,RainX Rain Repellant,5.0,United Kingdom,Good,0,2019-12-06
7578,RainX Rain Repellant,5.0,United Kingdom,"Have used this product before, it is excellent",0,2020-04-22


In [104]:
df_covid_dates.insert(1, 'Category', 'Auto')
df_covid_dates.insert(2, 'Price', 0)
df_covid_dates = df_covid_dates[["Product", "Category", "Price", "Individual Review", "Location", "Date", "Review", "Helpful"]]
df_covid_dates

,Product,Category,Price,Individual Review,Location,Date,Review,Helpful
0,Dr. Beckmann Carpet Stain Remover | Removes ne...,Auto,0,5.0,United Kingdom,2020-01-13,I have used this product for a year or two and...,10
1,Bosch Headlight Bulb,Auto,0,4.0,United Kingdom,2020-04-17,Bought these after buying so called brighter l...,6
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,Auto,0,5.0,United Kingdom,2020-05-30,Was so terrified cos I spilled an unbelievably...,86
3,Amazon Basics Microfibre Cleaning Cloths Pack ...,Auto,0,5.0,United Kingdom,2020-06-14,I often pick up Amazon branded products for ho...,5
4,Amazon Basics Microfibre Cleaning Cloths Pack ...,Auto,0,4.0,United Kingdom,2020-05-30,"Summary: Excellent Quality, a bit pricey.\n\nP...",0
...,...,...,...,...,...,...,...,...
7575,RainX Rain Repellant,Auto,0,5.0,United Kingdom,2019-12-22,Works perfect for my windscreen.,0
7576,RainX Rain Repellant,Auto,0,5.0,United Kingdom,2019-12-08,V Good,0
7577,RainX Rain Repellant,Auto,0,5.0,United Kingdom,2019-12-06,Good,0
7578,RainX Rain Repellant,Auto,0,5.0,United Kingdom,2020-04-22,"Have used this product before, it is excellent",0


In [105]:
df_covid_dates

,Product,Category,Price,Individual Review,Location,Date,Review,Helpful
0,Dr. Beckmann Carpet Stain Remover | Removes ne...,Auto,0,5.0,United Kingdom,2020-01-13,I have used this product for a year or two and...,10
1,Bosch Headlight Bulb,Auto,0,4.0,United Kingdom,2020-04-17,Bought these after buying so called brighter l...,6
2,Dr. Beckmann Carpet Stain Remover | Removes ne...,Auto,0,5.0,United Kingdom,2020-05-30,Was so terrified cos I spilled an unbelievably...,86
3,Amazon Basics Microfibre Cleaning Cloths Pack ...,Auto,0,5.0,United Kingdom,2020-06-14,I often pick up Amazon branded products for ho...,5
4,Amazon Basics Microfibre Cleaning Cloths Pack ...,Auto,0,4.0,United Kingdom,2020-05-30,"Summary: Excellent Quality, a bit pricey.\n\nP...",0
...,...,...,...,...,...,...,...,...
7575,RainX Rain Repellant,Auto,0,5.0,United Kingdom,2019-12-22,Works perfect for my windscreen.,0
7576,RainX Rain Repellant,Auto,0,5.0,United Kingdom,2019-12-08,V Good,0
7577,RainX Rain Repellant,Auto,0,5.0,United Kingdom,2019-12-06,Good,0
7578,RainX Rain Repellant,Auto,0,5.0,United Kingdom,2020-04-22,"Have used this product before, it is excellent",0


In [107]:
df_covid_dates.isna().value_counts()

Product  Category  Price  Individual Review  Location  Date   Review  Helpful
False    False     False  False              False     False  False   False      7580
dtype: int64

In [106]:
df_covid_dates.to_csv('../Cleaned_&_Sentimental_csv/auto_for_sentimental.csv', index=False)